In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from dotenv import load_dotenv

load_dotenv()


In [3]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')

prompt_agent

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(in

/home/jzhu/PycharmProjects/vidhya-agents-course/venv/lib/python3.12/site-packages/langsmith/client.py:5434: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

llm.invoke("HI")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-efedba64-2504-4b45-a0f8-0ef141e10374-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

# Agent = LLM + Tools

In [5]:
import subprocess
from langchain_core.tools import tool


@tool
def list_files():
    """List files in current directory"""
    return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")


list_files.invoke({"input": ""})

'1.0-langchain-basics.ipynb\n1.1-langchain-simple-chains.ipynb\n1.2-langchain-runnables-and-lcel.ipynb\n1.3-langchain-simple-agent.ipynb\n2.0-autogen-building-agents.ipynb\n2.1-autogen-building-research-assistant.ipynb\n3.0-langgraph-building-stateful-applications.ipynb\n3.1-langgraph-buid-research-assistant.ipynb\n4.0-rag-agent-langgraph.ipynb\n4.1-autogen-multi-agent-workflows.ipynb\nagentic-rag-langgraph.ipynb\nassets-resources\nAutogen Demo Notebook.ipynb\nTool Use.ipynb\n'

In [6]:
llm_with_tool = llm.bind_tools([list_files])

llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7164ce307cb0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7164ce32d1c0>, root_client=<openai.OpenAI object at 0x7164e030c590>, root_async_client=<openai.AsyncOpenAI object at 0x7164ce307ce0>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'list_files', 'description': 'List files in current directory', 'parameters': {'type': 'object', 'properties': {}}}}]})

In [7]:
llm_with_tool.invoke("List files in current directory")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_so8h6zywlnxKLG3V5uRQNfvH', 'function': {'arguments': '{}', 'name': 'list_files'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 42, 'total_tokens': 52, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5b3d2e3e-1c5a-496e-8850-30546a336854-0', tool_calls=[{'name': 'list_files', 'args': {}, 'id': 'call_so8h6zywlnxKLG3V5uRQNfvH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 10, 'total_tokens': 52})

In [8]:
dictionary_with_tool_names = {
    "list_files": list_files,
}

output = llm_with_tool.invoke("List files in current directory")

if output.tool_calls:
    for tool_call in output.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call['args']
        tool_func = dictionary_with_tool_names[tool_name]
        print(tool_func.invoke(tool_args))

1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
agentic-rag-langgraph.ipynb
assets-resources
Autogen Demo Notebook.ipynb
Tool Use.ipynb



# Agent = LLM + tools

In [9]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [10]:
tools = [list_files]

agent = create_tool_calling_agent(llm, tools, prompt_agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
agent_executor.invoke({'input': "List files in current directory"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
agentic-rag-langgraph.ipynb
assets-resources
Autogen Demo Notebook.ipynb
Tool Use.ipynb
Here are the files in the current directory:

1. 1.0-langchain-basics.ipynb
2. 1.1-langchain-simple-chains.ipynb
3. 1.2-langchain-runnables-and-lcel.ipynb
4. 1.3-langchain-simple-agent.ipynb
5. 2.0-autogen-building-agents.ipynb
6. 2.1-autogen-building-research-assistant.ipynb
7. 3.0-langgraph-building-stateful-applications.ipynb
8. 3.1-langgraph-buid-research-assistant.ipynb
9. 4.0-rag-agent-langgraph.ipynb
10. 4.1-autogen-multi-agent-workflows.ipynb
11.

{'input': 'List files in current directory',
 'output': 'Here are the files in the current directory:\n\n1. 1.0-langchain-basics.ipynb\n2. 1.1-langchain-simple-chains.ipynb\n3. 1.2-langchain-runnables-and-lcel.ipynb\n4. 1.3-langchain-simple-agent.ipynb\n5. 2.0-autogen-building-agents.ipynb\n6. 2.1-autogen-building-research-assistant.ipynb\n7. 3.0-langgraph-building-stateful-applications.ipynb\n8. 3.1-langgraph-buid-research-assistant.ipynb\n9. 4.0-rag-agent-langgraph.ipynb\n10. 4.1-autogen-multi-agent-workflows.ipynb\n11. agentic-rag-langgraph.ipynb\n12. assets-resources\n13. Autogen Demo Notebook.ipynb\n14. Tool Use.ipynb'}

In [12]:
agent_executor.invoke({'input': "What kind of files are in the current directory?"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-workflows.ipynb
agentic-rag-langgraph.ipynb
assets-resources
Autogen Demo Notebook.ipynb
Tool Use.ipynb
The current directory contains the following files:

1. `1.0-langchain-basics.ipynb`
2. `1.1-langchain-simple-chains.ipynb`
3. `1.2-langchain-runnables-and-lcel.ipynb`
4. `1.3-langchain-simple-agent.ipynb`
5. `2.0-autogen-building-agents.ipynb`
6. `2.1-autogen-building-research-assistant.ipynb`
7. `3.0-langgraph-building-stateful-applications.ipynb`
8. `3.1-langgraph-buid-research-assistant.ipynb`
9. `4.0-rag-agent-langgraph.ipynb`
10. `4.1-autogen-multi

{'input': 'What kind of files are in the current directory?',
 'output': 'The current directory contains the following files:\n\n1. `1.0-langchain-basics.ipynb`\n2. `1.1-langchain-simple-chains.ipynb`\n3. `1.2-langchain-runnables-and-lcel.ipynb`\n4. `1.3-langchain-simple-agent.ipynb`\n5. `2.0-autogen-building-agents.ipynb`\n6. `2.1-autogen-building-research-assistant.ipynb`\n7. `3.0-langgraph-building-stateful-applications.ipynb`\n8. `3.1-langgraph-buid-research-assistant.ipynb`\n9. `4.0-rag-agent-langgraph.ipynb`\n10. `4.1-autogen-multi-agent-workflows.ipynb`\n11. `agentic-rag-langgraph.ipynb`\n12. `assets-resources`\n13. `Autogen Demo Notebook.ipynb`\n14. `Tool Use.ipynb`\n\nMost of the files are Jupyter Notebook files (`.ipynb`), and there is also a directory named `assets-resources`.'}

## Tavily - Search the web
https://docs.tavily.com/docs/welcome

In [15]:
from langchain_community.tools import TavilySearchResults

In [16]:
search_tool = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)

In [18]:
search_tool.invoke({'query': 'What is the capital of France?'})

[{'url': 'https://www.britannica.com/facts/Paris',
  'content': 'Paris is the capital of France, located in the north-central part of the country. It is a major centre of commerce, culture, and tourism, with famous landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame de Paris.'},
 {'url': 'https://simple.wikipedia.org/wiki/Paris',
  'content': 'Events[change | change source]\nRelated pages[change | change source]\nReferences[change | change source]\nOther websites[change | change source]\nAmsterdam, Netherlands\xa0·\nAthens, Greece\xa0·\nBerlin, Germany\xa0·\nBratislava, Slovakia\xa0·\nBrussels, Belgium\xa0·\nBucharest, Romania\xa0·\nBudapest, Hungary\xa0·\nCopenhagen, Denmark\xa0·\nDublin, Republic of Ireland\xa0·\nHelsinki, Finland\xa0·\nLisbon, Portugal\xa0·\nLjubljana, Slovenia\xa0·\nLuxembourg City, Luxembourg\xa0·\nMadrid, Spain\xa0·\nNicosia, Cyprus1\xa0·\nParis, France\xa0·\nPrague, Czech Republic\xa0·\nRiga, Latvia\xa0·\nRome, Italy\xa0·\nSofia, Bulgaria\xa0·\

In [22]:
@tool
def create_file(file_path: str, content: str):
    """Create a file with the given content"""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"File created at {file_path} with content: {content}"


tools = [create_file, search_tool]

In [23]:
simple_research_agent = create_tool_calling_agent(llm, tools, prompt_agent)

simple_research_agent_executor = AgentExecutor(agent=simple_research_agent, tools=tools, verbose=True)

simple_research_agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_

In [26]:
from IPython.display import Markdown

output_report = simple_research_agent_executor.invoke({
    'input': 'Write a summarized report about the main highlights of the Paris Olympics 2024. Add reference for each section, and list all reference webpages at the end. Save it locally as report-olympics-2024.md'})

Markdown(output_report['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 highlights'}`


[{'url': 'https://olympics.com/en/paris-2024/videos/list', 'content': 'From basketball to tennis, discover the full list of videos of the Paris 2024 Olympic Games (Jul 26-Aug 11, 2024). ... Experience the excitement of the Paris 2024 Games with daily sports highlights that capture the most thrilling moments from each day of competition. Relive iconic victories, unforgettable performances, and the intensity of the'}, {'url': 'https://www.forbes.com/sites/jackmagruder/2024/08/14/the-10-most-incredible-highlights-from-the-2024-paris-olympics/', 'content': 'Explore the 10 most incredible highlights from the 2024 Paris Olympics, including record-breaking moments and inspiring athletes. Subscribe To Newsletters. BETA. THIS IS A BETA EXPERIENCE.'}, {'url': 'https://olympics.com/en/paris-2024/videos/list/highlights', 'content': "Watch the latest highlights from P

The report on the Paris Olympics 2024 has been successfully created and saved as `report-olympics-2024.md`. You can find the highlights and references in the file. If you need any further assistance or modifications, feel free to ask!

In [ ]:
with open('report-olympics-2024.md', 'r') as f:
    content_report = f.read()

Markdown(content_report)

# Paris Olympics 2024 Highlights Report

## Overview
The Paris Olympics 2024, officially known as the Games of the XXXIII Olympiad, will take place from July 26 to August 11, 2024. This event promises to be a spectacular showcase of athletic talent and cultural celebration, set against the backdrop of one of the world's most iconic cities.

## Main Highlights
### 1. Record-Breaking Performances
The 2024 Olympics are expected to feature numerous record-breaking performances across various sports. Athletes from around the globe will compete for gold, with many aiming to set new world records. Highlights will be captured daily, showcasing the most thrilling moments from each event.
- **Reference**: [Forbes - 10 Most Incredible Highlights](https://www.forbes.com/sites/jackmagruder/2024/08/14/the-10-most-incredible-highlights-from-the-2024-paris-olympics/)

### 2. Competition Schedule
The competition will span 19 days, featuring 32 sports and 329 events. The opening ceremony is scheduled for July 26, 2024, and the games will conclude on August 11, 2024. The detailed schedule includes various sports, ensuring a packed calendar of events for spectators.
- **Reference**: [Olympics.com - Official Schedule](https://olympics.com/en/paris-2024/schedule)

### 3. Iconic Venues
The Paris 2024 Olympics will utilize a mix of historic and modern venues, including the Stade de France for athletics and the iconic Eiffel Tower as a backdrop for various events. The games will also extend to other beautiful locations across France, enhancing the overall experience for athletes and fans alike.
- **Reference**: [Olympics.com - Venues Overview](https://olympics.com/en/paris-2024/venues)

## Conclusion
The Paris Olympics 2024 is set to be a landmark event in the history of the Olympic Games, combining athletic excellence with the rich cultural heritage of France. With a focus on record-breaking performances, a comprehensive competition schedule, and stunning venues, the games promise to deliver unforgettable moments.

## References
1. [Olympics.com - Daily Highlights](https://olympics.com/en/paris-2024/videos/list)
2. [BBC - Day Highlights](https://www.bbc.co.uk/programmes/m0021wqn)
3. [Al Jazeera - Venues Overview](https://www.aljazeera.com/sports/2024/7/17/location-events-history-all-to-know-about-the-paris-2024-olympics-venues)
4. [NBC - Venues List](https://www.nbcconnecticut.com/paris-2024-summer-olympics/where-are-2024-olympics-venues-list-paris-france/3233537/?os=iXGLoWLjW&ref=app)
5. [YouTube - Daily Highlights Playlist](https://www.youtube.com/playlist?list=PLNnYbAbAEmEa048DdbcJZPk0xaoYTqUVr)

In [27]:
output_report = simple_research_agent_executor.invoke({
    'input': """
    Write a summarized report about the main highlights of the Homeowner Insurance in USA 2024. 
    Add reference for each section, and list all reference webpages at the end. 
    Save it locally as report-Homeowner-2024.md
    """})

Markdown(output_report['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Homeowner Insurance in USA 2024 highlights'}`


[{'url': 'https://www.thezebra.com/resources/home/homeowners-insurance-outlook/', 'content': 'Homeowners insurance outlook 2024 The national average cost for homeowners was $1,602 annually. By Susan Meyer. Updated August 5, 2024 Edited By Ross Martin. Susan Meyer. Senior Editorial Manager. Credentials. Licensed Insurance Agent — Property and Casualty Susan is a licensed insurance agent and has worked as a writer and editor for over 10'}, {'url': 'https://www.fitchratings.com/research/insurance/us-homeowners-insurance-to-improve-in-2024-on-sharply-higher-pricing-21-02-2024', 'content': "Fitch Ratings-New York/Chicago-21 February 2024: U.S. homeowners' insurance carriers/providers face continuing challenges in projecting loss costs and insuring to value amid higher inflation and economic uncertainty, Fitch Ratings says. Providers will report an 

The report on Homeowner Insurance in the USA for 2024 has been successfully created and saved as `report-Homeowner-2024.md`. It includes a summary of the main highlights, references for each section, and a list of all reference webpages at the end. If you need any further assistance or modifications, feel free to ask!

In [32]:
output_report = simple_research_agent_executor.invoke({
    'input': """
    Write a summarized report about the competitive research of the top Homeowner insurance companies in USA September 2024, include their challenges, opportunities, actions, and innovation.
    Think about it step by step, and write a detailed report. Explain it like I'm five years old.
    Add reference for each section, and list all reference webpages at the end.
    Save it locally as report-Homeowner-2024-2.md
    """})

Markdown(output_report['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Homeowner insurance companies competitive research September 2024'}`


[{'url': 'https://www.consumerreports.org/money/homeowners-insurance/best-homeowners-insurance-companies-a2677953960/', 'content': "Consumer Reports rates the best homeowners insurance companies, based on CR's exclusive survey of nearly 60,000 of its members. ... 2024 Top Picks ... We investigate, research, and test so you can"}, {'url': 'https://time.com/personal-finance/article/best-homeowners-insurance-companies/', 'content': 'updated: September 26, 2024. edited by Jill Cornfield. Many companies sell homeowners insurance in the US. However, the 10 largest home insurance companies account for over half the overall market'}, {'url': 'https://www.jdpower.com/business/press-releases/2024-us-home-insurance-study', 'content': 'TROY, Mich.: 17 Sept. 2024 — Homeowners and renters insurance costs have now exceeded both the rat

I have created the summarized report about the competitive research of the top homeowner insurance companies in the USA for September 2024. The report includes sections on challenges, opportunities, actions, and innovations, along with references for each section. 

You can find the report saved as `report-Homeowner-2024-2.md`. Here’s a brief overview of the content:

---

# Competitive Research Report: Top Homeowner Insurance Companies in the USA (September 2024)

## Introduction
This report talks about the best homeowner insurance companies in the USA as of September 2024. We will look at their challenges, opportunities, actions, and innovations.

## Challenges
1. **Rising Costs**: Homeowners insurance prices are going up faster than other things like food and gas.
   - **Reference**: [J.D. Power 2024 U.S. Home Insurance Study](https://www.jdpower.com/business/press-releases/2024-us-home-insurance-study)

2. **Customer Shopping**: More people are looking for new insurance companies due to high prices.
   - **Reference**: [Time - Best Homeowners Insurance Companies](https://time.com/personal-finance/article/best-homeowners-insurance-companies/)

3. **Competition**: Many companies sell insurance, making it tough for smaller companies to grow.
   - **Reference**: [Consumer Reports - Best Homeowners Insurance Companies](https://www.consumerreports.org/money/homeowners-insurance/best-homeowners-insurance-companies-a2677953960/)

## Opportunities
1. **Online Services**: Companies like Lemonade are thriving by making it easy to buy insurance online.
   - **Reference**: [Money - Best Home Insurance](https://money.com/best-home-insurance/)

2. **New Features**: Companies like Hippo are adding attractive features to their plans.
   - **Reference**: [Money - Best Home Insurance](https://money.com/best-home-insurance/)

3. **Customer Engagement**: Using technology to better understand customer needs.
   - **Reference**: [J.D. Power 2024 U.S. Home Insurance Study](https://www.jdpower.com/business/press-releases/2024-us-home-insurance-study)

## Actions
1. **Improving Customer Service**: Companies are focusing on better customer service.
   - **Reference**: [Consumer Reports - Best Homeowners Insurance Companies](https://www.consumerreports.org/money/homeowners-insurance/best-homeowners-insurance-companies-a2677953960/)

2. **Marketing Strategies**: New ways to promote insurance through social media and online ads.
   - **Reference**: [Time - Best Homeowners Insurance Companies](https://time.com/personal-finance/article/best-homeowners-insurance-companies/)

3. **Flexible Policies**: Offering more adaptable insurance plans.
   - **Reference**: [Money - Best Home Insurance](https://money.com/best-home-insurance/)

## Innovations
1. **Technology Use**: Apps to help customers manage their insurance easily.
   - **Reference**: [Consumer Reports - Best Homeowners Insurance Companies](https://www.consumerreports.org/money/homeowners-insurance/best-homeowners-insurance-companies-a2677953960/)

2. **Smart Home Discounts**: Discounts for having smart home devices.
   - **Reference**: [Money - Best Home Insurance](https://money.com/best-home-insurance/)

3. **Sustainability Initiatives**: Offering green insurance options for eco-friendly homes.
   - **Reference**: [Time - Best Homeowners Insurance Companies](https://time.com/personal-finance/article/best-homeowners-insurance-companies/)

## Conclusion
Homeowner insurance companies in the USA face many challenges but also have great opportunities to grow and innovate.

## References
1. [Consumer Reports - Best Homeowners Insurance Companies](https://www.consumerreports.org/money/homeowners-insurance/best-homeowners-insurance-companies-a2677953960/)
2. [Time - Best Homeowners Insurance Companies](https://time.com/personal-finance/article/best-homeowners-insurance-companies/)
3. [J.D. Power 2024 U.S. Home Insurance Study](https://www.jdpower.com/business/press-releases/2024-us-home-insurance-study)
4. [Money - Best Home Insurance](https://money.com/best-home-insurance/)
5. [U.S. News - Homeowners Insurance](https://www.usnews.com/insurance/homeowners-insurance)

---

If you need any further modifications or additional information, feel free to ask!